<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/138_B2B_Sales_Agent_Claude_03_Personalization_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Personalization Agent

In [ ]:
"""
Personalization Agent - Creates custom outreach messages based on analysis

This agent demonstrates:
- Template-based content generation
- Personalization logic and customization
- Multi-channel message creation
- Tone and style adaptation
"""

import logging
from typing import Dict, List, Optional
from dataclasses import dataclass
import json
from research_agent import CompanyInfo
from analysis_agent import AnalysisResult

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class OutreachMessage:
    """Data structure for personalized outreach messages"""
    channel: str  # "email", "linkedin", "phone", "social"
    subject: str
    body: str
    tone: str  # "professional", "casual", "urgent", "consultative"
    personalization_elements: List[str]  # What was personalized
    call_to_action: str

@dataclass
class PersonalizationResult:
    """Complete personalization result"""
    company_name: str
    primary_contact: Optional[Dict[str, str]]
    messages: List[OutreachMessage]
    personalization_strategy: str
    recommended_sequence: List[str]  # Recommended order of outreach

class PersonalizationAgent:
    """
    Personalization Agent that creates custom outreach messages

    In a real implementation, this would:
    - Use LLMs for natural language generation
    - Integrate with CRM systems for contact preferences
    - Use A/B testing frameworks for message optimization
    - Leverage behavioral data for personalization
    """

    def __init__(self, agent_id: str = "personalization_agent"):
        self.agent_id = agent_id
        self.logger = logging.getLogger(f"{__name__}.{agent_id}")

        # Message templates for different scenarios
        self._email_templates = {
            "pain_point_focused": {
                "subject": "Quick question about {pain_point} at {company_name}",
                "body": """Hi {contact_name},

I noticed that {company_name} recently {recent_news}. This caught my attention because many {industry} companies are facing similar {pain_point} challenges.

I've been working with companies like yours to {solution_description}, and I thought you might find this interesting.

Would you be open to a brief 15-minute conversation about how other {industry} leaders are addressing {pain_point}?

Best regards,
{sender_name}""",
                "tone": "consultative"
            },
            "opportunity_focused": {
                "subject": "{company_name} growth opportunity - quick question",
                "body": """Hi {contact_name},

Congratulations on {recent_news}! It's exciting to see {company_name} {opportunity_description}.

I work with {industry} companies to {solution_description}, and I believe there might be a significant opportunity for {company_name} to {opportunity_value}.

Would you be interested in a brief conversation about how similar companies are {opportunity_category}?

Best regards,
{sender_name}""",
                "tone": "professional"
            },
            "relationship_building": {
                "subject": "Quick introduction from a fellow {industry} professional",
                "body": """Hi {contact_name},

I hope this email finds you well. I've been following {company_name}'s progress in the {industry} space, particularly your work on {company_description}.

I'm reaching out because I work with {industry} companies to {solution_description}, and I'd love to learn more about {company_name}'s approach to {industry_challenge}.

Would you be open to a brief conversation about industry trends and best practices?

Best regards,
{sender_name}""",
                "tone": "casual"
            }
        }

        self._linkedin_templates = {
            "connection_request": "Hi {contact_name}, I'd love to connect and learn more about {company_name}'s work in {industry}. Best regards, {sender_name}",
            "follow_up_message": "Thanks for connecting! I noticed {company_name} recently {recent_news}. I work with {industry} companies on {solution_area} - would love to share some insights that might be relevant."
        }

    def personalize_outreach(self, company_info: CompanyInfo, analysis_result: AnalysisResult,
                           sender_name: str = "Sales Professional") -> PersonalizationResult:
        """
        Create personalized outreach messages based on company research and analysis

        Args:
            company_info: CompanyInfo from Research Agent
            analysis_result: AnalysisResult from Analysis Agent
            sender_name: Name of the person sending the outreach

        Returns:
            PersonalizationResult with customized messages

        Raises:
            ValueError: If inputs are invalid
        """
        if not company_info or not analysis_result:
            raise ValueError("Valid CompanyInfo and AnalysisResult required")

        self.logger.info(f"Creating personalized outreach for {company_info.name}")

        try:
            # Determine personalization strategy
            strategy = self._determine_strategy(analysis_result)

            # Select primary contact
            primary_contact = self._select_primary_contact(company_info, analysis_result)

            # Create messages for different channels
            messages = self._create_messages(company_info, analysis_result, primary_contact, sender_name, strategy)

            # Determine recommended sequence
            sequence = self._determine_sequence(messages, analysis_result)

            result = PersonalizationResult(
                company_name=company_info.name,
                primary_contact=primary_contact,
                messages=messages,
                personalization_strategy=strategy,
                recommended_sequence=sequence
            )

            self.logger.info(f"Created {len(messages)} personalized messages for {company_info.name}")
            return result

        except Exception as e:
            self.logger.error(f"Error personalizing outreach for {company_info.name}: {str(e)}")
            raise

    def _determine_strategy(self, analysis_result: AnalysisResult) -> str:
        """Determine the best personalization strategy"""
        high_severity_pains = [p for p in analysis_result.pain_points if p.severity in ["high", "critical"]]
        high_priority_opps = [o for o in analysis_result.opportunities if o.priority in ["high", "urgent"]]

        if high_severity_pains:
            return "pain_point_focused"
        elif high_priority_opps:
            return "opportunity_focused"
        else:
            return "relationship_building"

    def _select_primary_contact(self, company_info: CompanyInfo, analysis_result: AnalysisResult) -> Optional[Dict[str, str]]:
        """Select the best contact for outreach"""
        if not company_info.key_contacts:
            return None

        # Simple logic: prefer CEO for startups, VP/CTO for larger companies
        if company_info.size == "startup":
            for contact in company_info.key_contacts:
                if "CEO" in contact.get("title", "").upper() or "Founder" in contact.get("title", "").upper():
                    return contact

        # For larger companies, prefer operational roles
        for contact in company_info.key_contacts:
            if any(role in contact.get("title", "").upper() for role in ["VP", "CTO", "COO", "DIRECTOR"]):
                return contact

        # Fallback to first contact
        return company_info.key_contacts[0]

    def _create_messages(self, company_info: CompanyInfo, analysis_result: AnalysisResult,
                       primary_contact: Optional[Dict[str, str]], sender_name: str, strategy: str) -> List[OutreachMessage]:
        """Create personalized messages for different channels"""
        messages = []

        if not primary_contact:
            self.logger.warning(f"No contact information available for {company_info.name}")
            return messages

        contact_name = primary_contact.get("name", "there")

        # Create email message
        email_message = self._create_email_message(company_info, analysis_result, primary_contact, sender_name, strategy)
        if email_message:
            messages.append(email_message)

        # Create LinkedIn message
        linkedin_message = self._create_linkedin_message(company_info, analysis_result, primary_contact, sender_name)
        if linkedin_message:
            messages.append(linkedin_message)

        return messages

    def _create_email_message(self, company_info: CompanyInfo, analysis_result: AnalysisResult,
                            primary_contact: Dict[str, str], sender_name: str, strategy: str) -> OutreachMessage:
        """Create personalized email message"""
        template = self._email_templates.get(strategy, self._email_templates["relationship_building"])

        # Prepare template variables
        variables = {
            "contact_name": primary_contact.get("name", "there"),
            "company_name": company_info.name,
            "industry": company_info.industry.lower(),
            "sender_name": sender_name,
            "company_description": company_info.description,
            "recent_news": analysis_result.company_name,  # Simplified for demo
            "industry_challenge": "operational efficiency"  # Simplified for demo
        }

        # Add strategy-specific variables
        if strategy == "pain_point_focused" and analysis_result.pain_points:
            pain_point = analysis_result.pain_points[0]
            variables.update({
                "pain_point": pain_point.description.lower(),
                "solution_description": pain_point.potential_solution.lower()
            })
        elif strategy == "opportunity_focused" and analysis_result.opportunities:
            opportunity = analysis_result.opportunities[0]
            variables.update({
                "opportunity_description": opportunity.description.lower(),
                "opportunity_value": opportunity.potential_value.lower(),
                "opportunity_category": opportunity.category,
                "solution_description": "growth strategies"
            })
        else:
            variables.update({
                "solution_description": "operational optimization",
                "solution_area": "process improvement"
            })

        # Format the message
        subject = template["subject"].format(**variables)
        body = template["body"].format(**variables)

        return OutreachMessage(
            channel="email",
            subject=subject,
            body=body,
            tone=template["tone"],
            personalization_elements=[
                f"Contact name: {variables['contact_name']}",
                f"Company-specific pain point: {variables.get('pain_point', 'N/A')}",
                f"Industry context: {variables['industry']}",
                f"Recent company news: {variables['recent_news']}"
            ],
            call_to_action="Schedule a 15-minute conversation"
        )

    def _create_linkedin_message(self, company_info: CompanyInfo, analysis_result: AnalysisResult,
                               primary_contact: Dict[str, str], sender_name: str) -> OutreachMessage:
        """Create LinkedIn connection request message"""
        contact_name = primary_contact.get("name", "there")

        # Use connection request template
        template = self._linkedin_templates["connection_request"]

        variables = {
            "contact_name": contact_name,
            "company_name": company_info.name,
            "industry": company_info.industry.lower(),
            "sender_name": sender_name
        }

        body = template.format(**variables)

        return OutreachMessage(
            channel="linkedin",
            subject="Connection Request",
            body=body,
            tone="professional",
            personalization_elements=[
                f"Contact name: {contact_name}",
                f"Company: {company_info.name}",
                f"Industry: {company_info.industry}"
            ],
            call_to_action="Connect and start conversation"
        )

    def _determine_sequence(self, messages: List[OutreachMessage], analysis_result: AnalysisResult) -> List[str]:
        """Determine recommended outreach sequence"""
        sequence = []

        # LinkedIn connection first
        linkedin_msg = next((msg for msg in messages if msg.channel == "linkedin"), None)
        if linkedin_msg:
            sequence.append("linkedin_connection")

        # Email follow-up
        email_msg = next((msg for msg in messages if msg.channel == "email"), None)
        if email_msg:
            sequence.append("email_outreach")

        # Additional follow-up based on confidence
        if analysis_result.confidence_score > 0.7:
            sequence.append("phone_follow_up")

        return sequence

    def get_status(self) -> Dict[str, str]:
        """Return agent status for monitoring"""
        return {
            "agent_id": self.agent_id,
            "status": "ready",
            "available_templates": list(self._email_templates.keys()),
            "supported_channels": ["email", "linkedin"]
        }

# Example usage and testing
if __name__ == "__main__":
    # Import and test with other agents
    from research_agent import ResearchAgent
    from analysis_agent import AnalysisAgent

    print("=== Personalization Agent Demo ===\n")

    # Create agents
    research_agent = ResearchAgent()
    analysis_agent = AnalysisAgent()
    personalization_agent = PersonalizationAgent()

    # Get company info and analysis
    company_info = research_agent.research_company("Acme Corporation")

    if company_info:
        analysis_result = analysis_agent.analyze_company(company_info)
        personalization_result = personalization_agent.personalize_outreach(
            company_info, analysis_result, "John Smith"
        )

        print(f"Personalization for: {personalization_result.company_name}")
        print(f"Strategy: {personalization_result.personalization_strategy}")
        print(f"Primary Contact: {personalization_result.primary_contact}")

        print(f"\nMessages ({len(personalization_result.messages)}):")
        for i, message in enumerate(personalization_result.messages, 1):
            print(f"\n{i}. {message.channel.upper()} Message:")
            print(f"   Subject: {message.subject}")
            print(f"   Tone: {message.tone}")
            print(f"   Body Preview: {message.body[:100]}...")
            print(f"   Personalized Elements: {', '.join(message.personalization_elements)}")

        print(f"\nRecommended Sequence:")
        for i, step in enumerate(personalization_result.recommended_sequence, 1):
            print(f"  {i}. {step}")

    print("\n" + "="*50 + "\n")

    # Show agent status
    status = personalization_agent.get_status()
    print(f"Agent Status: {json.dumps(status, indent=2)}")


# Personalization Agent Code Explained
Let’s break down the **Personalization Agent** and highlight what’s worth focusing on.

---

## 🔹 Purpose of the Personalization Agent

This agent takes the structured data from:

* **ResearchAgent** (`CompanyInfo`) → facts about the company.
* **AnalysisAgent** (`AnalysisResult`) → pain points, opportunities, confidence score.

…and turns them into **personalized outreach messages** across multiple channels (email, LinkedIn), with a recommended sequence.

👉 In other words: it’s the *last-mile agent* that converts analysis into real sales actions.

---

## 🔹 Key Things to Focus On

### 1. **Data Models**

* `OutreachMessage`: A structured container for each outreach attempt (channel, subject, body, tone, personalization elements, CTA).
* `PersonalizationResult`: Wraps everything into a single deliverable — company name, chosen contact, all messages, strategy, and sequence.

💡 *Lesson*: Structured data is critical in multi-agent systems. This ensures downstream agents or tools (like CRM integrators) can consume outputs reliably.

---

### 2. **Personalization Strategy**

Method: `_determine_strategy()`

* Chooses between **pain-point-focused**, **opportunity-focused**, or **relationship-building** messaging.
* Decision based on severity of pain points or priority of opportunities.

💡 *Lesson*: Even if LLMs generate text, the *strategy logic* should be deterministic and explainable. This agent uses clear rules before handing off to templating/NLG.

---

### 3. **Contact Selection**

Method: `_select_primary_contact()`

* Prefers **CEO/Founder** for startups.
* Prefers **VP/CTO/COO/Director** for larger companies.
* Falls back to the first contact if no clear match.

💡 *Lesson*: Role-based heuristics make personalization smarter. It’s not just what you say, it’s *who you say it to*.

---

### 4. **Templates and Channels**

* Email templates for different strategies (pain point, opportunity, relationship).
* LinkedIn templates (connection request, follow-up).
* Templating uses simple Python `.format()` with injected variables.

💡 *Lesson*: Templates keep messaging **consistent, scalable, and editable**. The system separates *logic* (deciding what to say) from *content* (how it’s said).

---

### 5. **Multi-Channel Outreach**

* Creates both an **email message** and a **LinkedIn message** for each company.
* Each channel message is wrapped in an `OutreachMessage` dataclass.

💡 *Lesson*: Multi-channel presence is a best practice in sales orchestration. Build once → deploy to email, LinkedIn, phone.

---

### 6. **Outreach Sequence**

Method: `_determine_sequence()`

* Starts with a LinkedIn connection.
* Follows with an email outreach.
* Adds a phone follow-up if confidence score is high.

💡 *Lesson*: Sequence matters in sales! This agent encodes sales playbook logic into software.

---

### 7. **Agent Observability**

Method: `get_status()`

* Returns a small health/status report (agent\_id, status, templates available, supported channels).

💡 *Lesson*: Status endpoints are simple but powerful — you can query each agent to monitor availability/configuration without running a full workflow.

---

## 🔹 What Jumps Out

1. **Templates as first-class citizens** → This agent is designed to scale. Non-engineers (sales ops, marketers) could swap templates without touching the orchestration code.
2. **Deterministic strategy logic** → Instead of dumping everything into an LLM, this agent makes key targeting decisions with rules. That makes it **transparent and auditable**.
3. **Structured, channel-specific outputs** → Messages are neatly packaged so they can flow into a CRM, email sender, or LinkedIn automation tool without extra parsing.

---

## ✅ What You Should Take Away

* **Focus**: Learn how data flows from research → analysis → personalization, and how each agent adds structure.
* **Practice**: Notice how deterministic heuristics (role matching, pain vs. opportunity strategy) complement LLM-based personalization.
* **Principle**: Keep personalization modular (templates, channels, strategies) so it can evolve without rewriting the agent.





Right now, your **PersonalizationAgent** is mostly:

* Using **pre-written templates**
* Injecting variables from research/analysis
* Choosing a **strategy** (pain-point, opportunity, or relationship)

That’s a great MVP, but it *caps out* because the personalization is only as good as the static templates.

---

## 🚀 Why a TemplateCraftingAgent is a Strong Idea

1. **Dynamic Template Generation**

   * Instead of pulling from a fixed set, the agent can **craft a bespoke template** using context:

     * Research findings
     * Pain points & opportunities
     * SWOT analysis (strengths, weaknesses, opportunities, threats)
   * That means *every message* could be subtly different — higher chance of engagement.

2. **LLM Strengths**

   * LLMs are very good at *pattern expansion* and *stylistic variation*.
   * Example: for a SaaS company in finance, it might generate a consultative style; for a manufacturing firm, more operational language.

3. **Human-in-the-Loop Ready**

   * You could store a “library” of crafted templates and let humans pick the best. Over time, you build your own dataset of high-performing outreach.

4. **Scalability**

   * Sales teams often need fresh messaging for each campaign or vertical. Automating template creation offloads a lot of copywriting overhead.

---

## 🔹 How it Could Fit into Your Pipeline

```
ResearchAgent  →  AnalysisAgent  →  TemplateCraftingAgent  →  PersonalizationAgent
```

* **TemplateCraftingAgent** → generates raw bespoke templates (maybe 2–3 variations).
* **PersonalizationAgent** → applies company/contact data, finalizes CTA, and formats across channels (email, LinkedIn, etc).

This way:

* One agent creates **message skeletons**.
* The other agent ensures **channel-ready, structured outputs**.

---

## 🔹 Key Design Considerations

* **Prompts & Guardrails**: You’ll want to control tone, length, and compliance (no overpromises).
* **Reusability**: Save generated templates in a repository (JSON or database) for future use.
* **A/B Testing**: Orchestrator could pick different templates for different leads to measure effectiveness.
* **Fallbacks**: If the LLM fails, fall back to a static template so the workflow doesn’t break.

---

## ✅ My Take

This idea upgrades your system from *“templated personalization”* to *“adaptive personalization”*. It leans into what LLMs do best (language creativity) while keeping deterministic structure (outreach sequence, contact targeting).





## 🔹 What is the `PersonalizationResult` class?

`PersonalizationResult` is a **dataclass** that defines the *structured output* of your **PersonalizationAgent**.
Instead of dumping free-form text, it returns a **container** that neatly organizes all the relevant personalization details.

Think of it as the *package* your agent hands back to the orchestrator.

---

## 🔹 Fields Explained

```python
@dataclass
class PersonalizationResult:
    """Complete personalization result"""
    company_name: str
    primary_contact: Optional[Dict[str, str]]
    messages: List[OutreachMessage]
    personalization_strategy: str
    recommended_sequence: List[str]  # Recommended order of outreach
```

1. **`company_name: str`**

   * The company this personalization is about.
   * Example: `"Acme Corp"`

2. **`primary_contact: Optional[Dict[str, str]]`**

   * The main decision-maker to target.
   * It’s optional (`Optional[...]`) because sometimes no good contact is found.
   * Typically includes fields like:

     ```python
     {"name": "Jane Doe", "role": "VP of Engineering", "email": "jane@acme.com"}
     ```

3. **`messages: List[OutreachMessage]`**

   * A list of structured outreach messages created for different channels.
   * Each message is an `OutreachMessage` dataclass (likely with fields like `channel`, `subject`, `body`, `tone`, etc).
   * Example:

     * Email outreach
     * LinkedIn message
     * Phone script

4. **`personalization_strategy: str`**

   * The high-level *approach* chosen for this lead.
   * Example values: `"pain-point-focused"`, `"opportunity-focused"`, `"relationship-building"`.
   * Useful for reporting, auditing, and debugging why a certain style of messaging was chosen.

5. **`recommended_sequence: List[str]`**

   * The playbook for outreach order.
   * Example:

     ```python
     ["linkedin_connection", "email_outreach", "phone_followup"]
     ```
   * Helps orchestrators and CRM integrations know *when and how* to deploy messages.

---

## 🔹 Why It’s Important

* **Structure > Chaos** → Instead of just text blobs, you now have clean, structured data that can:

  * Feed into a CRM
  * Be logged and analyzed
  * Be tested systematically

* **Agent-Orchestrator Friendly** → Other agents (like a sending agent or performance tracker) can consume this without guessing where fields live.

* **Extensible** → You can add more fields later (e.g., confidence score, variant\_id for A/B testing, timestamp).

---

✅ **In short:**
`PersonalizationResult` is the **output contract** for your PersonalizationAgent. It defines what “done” looks like:

* Who to contact
* What messages to send
* What strategy was used
* In what order to reach out





## 🔹 Why Structured Returns Matter for LLMs

1. **Reduces Ambiguity**

   * If you just ask an LLM: *“Write a personalized outreach plan”*, it might give you a beautiful wall of text.
   * But your orchestrator (or CRM system) won’t know which part is the company name, which is the email body, or which is the sequence.
   * By enforcing a schema (like `PersonalizationResult`), you eliminate guessing.

2. **Reliable Orchestration**

   * Your orchestrator can trust:

     * `messages` will always be a list of `OutreachMessage` objects.
     * `personalization_strategy` will always be a single string.
     * `recommended_sequence` is a list of steps, in order.
   * This makes it easy to route results into the next agent or system.

3. **Machine-Readable for Automation**

   * JSON/dict-like structures are *machine-friendly*.
   * That means they can be logged, tested, inserted into a database, or pushed into a CRM API **without extra parsing**.

4. **Error Handling & Debugging**

   * If an LLM strays and outputs garbage, you can validate the structure (`pydantic`, JSON schema, dataclasses) before letting it propagate.
   * E.g., if `messages` isn’t a list → fail fast, retry, or fix.

5. **Consistency Across Runs**

   * Free-form text responses vary wildly between runs.
   * Structured schemas enforce predictability, even though the language inside (the outreach body) can vary.

---

## 🔹 Analogy

Think of it like restaurant orders:

* **Free-form output** → A waiter comes back and says *“The chef made you something nice”*.
* **Structured output** → The waiter hands you a printed ticket: *Main: Burger, Side: Fries, Drink: Coke*.

Which one would you rather pass to the kitchen robot that prepares trays?
Structured = no mistakes, no guessing.

---

✅ **In short:**
Yes, structured returns are critical for LLM pipelines. They’re how you make creative, fuzzy LLMs work inside precise, deterministic systems.


